# BYOC Inference for paddleOCR

We now want to use the model to perform inference, this includes: 

- Create model for the training outputCreate Endpoint Configuration 
- Create a configuration defining an endpoint.Create Endpoint 
- Use the configuration to create an inference endpoint.Perform Inference 
- Perform inference on some input data using the endpoint.

## Deploy model
we now create a SageMaker Model from the training output. Using the model we can create an Endpoint Configuration.


In [ ]:
%%time

from sagemaker import get_execution_role
role = get_execution_role()


import sagemaker
import boto3
from time import gmtime, strftime

sage = boto3.Session().client(service_name='sagemaker') 
sess = sagemaker.Session()

In [ ]:
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name

PROJECT_ID = "sagemaker-p-5an0os9jqfdi"

In [ ]:
hosting_image = f'{account}.dkr.ecr.{region}.amazonaws.com/{PROJECT_ID}-inference-imagebuild:latest'
print('Inference image location: ',hosting_image)

In [ ]:
# Replace with the correct previous training job name 
TrainingJobName = "Replace your own training job name"
TrainingJobName = "sagemaker-p-5an0os9jqfdi-training-image-2022-05-31-02-01-42-567"

In [ ]:
info = sage.describe_training_job(TrainingJobName=TrainingJobName)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print('Model artifacts location: ',model_data)

primary_container = {
    'Image': hosting_image,
    'ModelDataUrl': model_data,
}


In [ ]:
model_name="paddle-v1"

create_model_response = sage.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

## Create Endpoint Configuration

At launch, we will support configuring REST endpoints in hosting with multiple models, e.g. for A/B testing purposes. In order to support this, customers create an endpoint configuration, that describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way.In addition, the endpoint configuration describes the instance type required for model deployment, and at launch will describe the autoscaling configuration.


In [ ]:
from time import gmtime, strftime
import time 

job_name_prefix = "paddle"

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_config_name = job_name_prefix + '-epc-' + timestamp
endpoint_config_response = sage.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.p3.2xlarge',
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

##  Create Endpoint


In [ ]:
%%time
import time


timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_name = job_name_prefix + '-ep-' + timestamp
print('Endpoint name: {}'.format(endpoint_name))

endpoint_params = {
    'EndpointName': endpoint_name,
    'EndpointConfigName': endpoint_config_name,
}
endpoint_response = sage.create_endpoint(**endpoint_params)
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))

You can confirm the endpoint configuration and status by navigating to the "Endpoints" tab in the AWS SageMaker console.We will finally create a runtime object from which we can invoke the endpoint.

#  Perform Inference with image url 

In [ ]:
import boto3
from botocore.config import Config
from sagemaker.session import Session

config = Config(
    read_timeout=120,
    retries={
        'max_attempts': 0
    }
)

from boto3.session import Session
import json

In [ ]:
import sagemaker as sage
from time import gmtime, strftime
from sagemaker import get_execution_role

sess = sage.Session()
WORK_DIRECTORY = "./test/new_hkid_front.jpg"

# S3 prefix
prefix = "DEMO-paddle-byo"
bucket = sess.default_bucket()  

image_uri = f'{prefix}/new_hkid_front.jpg'

role = get_execution_role()

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)
print(data_location)

In [ ]:

test_data = {
    'bucket' : bucket,
    'image_uri' : image_uri,
    'content_type': "application/json",
}
payload = json.dumps(test_data)
print(payload)

In [17]:
sagemaker_runtime_client = boto3.client('sagemaker-runtime', config=config)
session = Session(sagemaker_runtime_client)

#     runtime = session.client("runtime.sagemaker",config=config)
response = sagemaker_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    #ContentType="image/jpeg",
    Body=payload)

result = json.loads(response["Body"].read())
print (result)

{'label': ['香港永久性居民身份', 'SAMP!', 'HONGKONGPERMANENTIDENTITYCARE', '而學永晴', 'Z683365', 'LOK，Wing', 'ching', 'MPLE', '2867', '3057', '2532', 'SAMPLE', '出生日期DateofBirth', 'Q3-06-1985', '女F', '大**AZ', 'SAMPLE', '發日期Dateoflssue', '（Q6-96)', '26-11-18', 'Z683365（5)'], 'confidences': [17.026626586914062, 17.13430404663086, 16.066162109375, 12.405200004577637, 16.365564346313477, 15.620736122131348, 16.953876495361328, 16.583984375, 18.67633056640625, 18.979454040527344, 20.221355438232422, 18.48903465270996, 15.502676963806152, 18.379587173461914, 13.910089492797852, 14.175936698913574, 17.855117797851562, 15.46212100982666, 15.202425003051758, 17.087268829345703, 16.568973541259766], 'bbox': [[[158.0, 10.0], [348.0, 11.0], [348.0, 31.0], [158.0, 30.0]], [[11.0, 21.0], [25.0, 21.0], [25.0, 66.0], [11.0, 66.0]], [[73.0, 36.0], [434.0, 36.0], [434.0, 52.0], [73.0, 52.0]], [[10.0, 63.0], [110.0, 67.0], [109.0, 91.0], [9.0, 87.0]], [[356.0, 65.0], [400.0, 65.0], [400.0, 78.0], [356.0, 78.0]], [[19

# Clean up
When we're done with the endpoint, we can just delete it and the backing instances will be released.  Run the following cell to delete the endpoint.

In [ ]:
print(endpoint_name)
sage.delete_endpoint(EndpointName=endpoint_name)
sage.delete_endpoint_config(EndpointConfigName=endpoint_name)
sage.delete_model(ModelName=endpoint_name)